# MSDS 610 — Final Project 2
### Eddie Flores

` **Note:** Optimized pkl model was too large to upload to Github.`

In [ ]:
# Reload necessary libraries
import pandas as pd
import numpy as np
import joblib

# Load the optimized model
optimized_model_path = "optimized_random_forest_model.pkl"
optimized_rf_model = joblib.load(optimized_model_path)

# Load the live data
live_data_path = "/mnt/data/live_data.csv"
df_live = pd.read_csv(live_data_path)

# Load the cleansing decisions table (assumed previously created)
cleansing_decisions_path = "/mnt/data/cleansing_decisions.csv"
df_cleansing_rules = pd.read_csv(cleansing_decisions_path)

# Display the first few rows of the live data to inspect its structure
df_live.head()
